# ML Trader

In [1]:
from utils.packages import *
from utils.variables import *
from utils.i_o import *
from utils.dir_slope import *
from utils.engine import *
from utils.loops import *
from utils.order import *
from utils.ml import *
import winsound

# Notes
---
## 2022_09_02

### Observations

1. No need to make 10 transactions a day. Just one is enough

1. lema or sema match
    1. Using lema match works fine in both opening 
    1. Using lema has better results
    1. 144 produces better resuls over higher numbers like 1440 

1. Lema angle in order and close
    1. Using lema angle in order, delays order, but overall profits are high
    1. Using lema angle in close also increases overall profit

1. Before everything need to find why 2021 is consistently producing negative results in all the setups.
    1. What is that this causing this issue, how can it be addressed with in the same system
    1. less complex method is essential
    

### To Test
1. Find whether the opening or closing that is causing the most issue in loss years
1. Test other years to understand how well this setup works.
1. Remove lema match check from closing. This is delaying bad trasactions
1. Test 5 round up on lema match. Transactions are opening in lema match gaps
1. Ideation
    1. Test lema Bollinger bands to understand flow direction
    1. Test overall flow direction

---

## 2022_09_02

### Observations
1. Opening and closing need to happen faster. Cant close after ema match
1. Lema angle helps save positions against frequent flips but delays closing and causes losses
1. Market is non-directional (Dec 2021 is a good example) Goes up and down, when algo tries to open and close its always too late
1. Need a faster method


### To Test
1. Should test plotting close as a line instead of candles. Perform match in close with wider window
1. Close chart needs to be analysed. This looks promising.

---

## Result by Years

| Year | Angle open | Angle close | Lema match open | Lema match close | Result type | pips  |
| ---- | ---------- | ----------- | --------------- | ---------------- | ----------- | ----- |
| 2022 | Yes        | No          | Yes             | Yes              | Profit      | 200   |
| 2021 | Yes        | No          | Yes             | Yes              | Loss        | \-800 |
| 2020 | Yes        | No          | Yes             | Yes              | Profit      | 800   |
| 2019 | Yes        | No          | Yes             | Yes              | Loss        | \-400 |
| 2018 | Yes        | No          | Yes             | Yes              | Profit      | 300   |
| 2017 | Yes        | No          | Yes             | Yes              | Profit      | 14    |    

data                = get_date_list(data)
data                = read_data(data)

In [2]:
%%time
data = run_engine(data)

pl   -0.01262
dtype: float64
--------------------------------------
            DateTime_frmt  order_side  close_type       pl
70626 2021-12-15 06:35:00         NaN  sema_close -0.00127
70792 2021-12-15 20:25:00         NaN  sema_close -0.00269
71257 2021-12-17 11:10:00         NaN  sema_close  0.00283
71518 2021-12-20 08:55:00         NaN  sema_close  0.00025
71879 2021-12-21 15:00:00         NaN  sema_close -0.00227
72070 2021-12-22 06:55:00         NaN  sema_close -0.00152
72114 2021-12-22 10:35:00         NaN  sema_close -0.00289
72380 2021-12-23 08:45:00         NaN  sema_close  0.00059
72718 2021-12-24 12:55:00         NaN  sema_close -0.00136
72944 2021-12-27 07:50:00         NaN  sema_close -0.00233
73029 2021-12-27 14:55:00         NaN  sema_close -0.00201
73310 2021-12-28 14:20:00         NaN  sema_close -0.00154
73589 2021-12-29 13:35:00         NaN  sema_close -0.00084
73774 2021-12-30 05:00:00         NaN  sema_close -0.00083
74115 2021-12-31 09:25:00         NaN  sema_clo

100%|██████████████████████████████████████| 532/532 [00:00<00:00, 1137.29it/s]

Wall time: 1min 53s


In [3]:
x = data['df_ohlc'].groupby(['month_val'])['pl'].sum()
print(x)
print('---------------------')
print(x.sort_values())
print('---------------------')
print(f'max : {x.idxmax()} : {max(x)}')
print(f'min : {x.idxmin()} : {min(x)}')
print('---------------------')
monthly_avg = np.round(data['df_ohlc']['pl'].sum()/len(set(data['df_ohlc']['month_val'])),4)
print(f'monthly_avg : {monthly_avg}')

month_val
1     0.00498
2     0.00618
3     0.02688
4    -0.01564
5     0.01472
6     0.01113
7    -0.01155
8     0.00096
9    -0.01571
10   -0.00803
11    0.01578
12   -0.04232
Name: pl, dtype: float64
---------------------
month_val
12   -0.04232
9    -0.01571
4    -0.01564
7    -0.01155
10   -0.00803
8     0.00096
1     0.00498
2     0.00618
6     0.01113
5     0.01472
11    0.01578
3     0.02688
Name: pl, dtype: float64
---------------------
max : 3 : 0.026879999999999994
min : 12 : -0.04232
---------------------
monthly_avg : -0.0011


In [4]:
data['lema_gap_pl_analysis_month'] = 8

data = lema_gap_pl_analysis(data)
data['lema_gap_pl_report']

,index,positive,negative,count_diff,positive_sum,negative_sum,sum_diff
5,0.000,1.0,0.0,1.0,0.00408,0.00000,0.00408
0,0.001,3.0,9.0,-6.0,0.00829,-0.01292,-0.00463
1,0.002,1.0,6.0,-5.0,0.00638,-0.00597,0.00041
3,0.003,1.0,1.0,0.0,0.00325,-0.00092,0.00233
2,0.004,0.0,1.0,-1.0,0.00000,-0.00014,-0.00014
4,0.005,1.0,0.0,1.0,0.00248,0.00000,0.00248


In [5]:
sys.exit()

SystemExit: 

In [ ]:
print(data['df_small'].groupby(['weekday', 'pl_type'])['pl_type'].count().unstack())
print('----------------------------------------------')
print(data['df_small'].groupby(['weekday', 'pl_type'])['pl'].sum().unstack())

In [ ]:
data['lema_gap_pl_analysis_month'] = 3

data = lema_gap_pl_analysis(data)
data['lema_gap_pl_report']

In [ ]:
y = data['df_ohlc']['pl'].sum()
x = data['df_ohlc'].groupby(['month_val'])['pl'].sum()
send_telegram_message(f'{data["input_year"]}')
send_telegram_message(f'{y}')
send_telegram_message(f'{x}')
send_telegram_message(f'-------------------')

In [ ]:
y = data['df_ohlc'][data['df_ohlc']['month_val'] == 6]
x = y.groupby(['date_val'])['pl'].sum()
print(x)
print('---------------------')
print(f'max : {x.idxmax()} : {max(x)}')
print(f'min : {x.idxmin()} : {min(x)}')
print('---------------------')

In [ ]:
data['df_ohlc'].tail()

In [ ]:
data['df_ohlc']['lema_gap'].describe()

In [ ]:
x = 5
data['df_ohlc'].iloc[-5:,x:x+15]

In [ ]:
data['df_small']['lema_gap'].describe()

In [ ]:
a = np.arange(0,20)
for b in a:
    x = int(round(b/5.0)*5.0)
    print(b, x)

In [ ]:
abs(all['negative_sum']) / all['positive_sum']

In [ ]:
df_small['order_side']

In [ ]:
data['df_small'].to_csv('data/temp.csv')

In [6]:
data["plot"] = True
data['plot_type'] = 'file'
data['reduce_plot'] = True
month = 12

# Monthly Chart -------------------
data['plot_start']  = f"{data['input_year']}-{month}-1"

if month == 12:
    data['plot_stop']   = f"{data['input_year']+1}-1-1"
else:
    data['plot_stop']   = f"{data['input_year']}-{month+1}-1"

# Year Chart ----------------------------
# data['plot_start']  = f"{data['input_year']}-1-1"
# data['plot_stop']   = f"{data['input_year']+1}-1-1"
# Year Chart ----------------------------

# data['things_to_plot'] = ['tick', 'ema', 'BBands', 'dots']
# data['things_to_plot'] = ['ema', 'BBands', 'dots', 'indicators']
# data['things_to_plot'] = ['ema', 'BBands', 'dots', 'cross']
# data['things_to_plot'] = ['ema', 'BBands', 'dots', 'positions', 'tip']
# data['things_to_plot'] = ['ema', 'BBands', 'dots', 'positions', 'lema_angle']
# data['things_to_plot'] = ['ema', 'sema_angle', 'slema_angle', 'lema_angle', 'dir']
# data['things_to_plot'] = ['ema', 'positions', 'lema_angle','slema_angle','sema_angle', 'rsi']
# data['things_to_plot'] = ['ema','positions', 'lema_angle','lema_diff', 'lema_max_min']
# data['things_to_plot'] = ['candles','sema', 'slema','positions','lema', 'lema_angle', 'lema_max_min', 'close_diff']
# data['things_to_plot'] = ['lema', 'tip', 'candles', 'lema_angle', 'lema_change']
data['things_to_plot'] = ['candles','sema', 'close','lema', 'lema_match', 'positions', 'lema_change']

plot_graph(data)

In [ ]:
data['df_ohlc']['lema_angle']

In [ ]:
x = 0
np.sign(data['df_ohlc']['lema_angle'].diff())[x:x+50]

generate_result_report(data)